In [1]:
import pandas as pd
import numpy as np

from IPython.display import HTML
import warnings
warnings.simplefilter('ignore')

%matplotlib notebook

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Нажмите чтобы отобразить/скрыть код."></form>''')

In [2]:
demand = np.array([30,40,30])
supply = np.array([15,15,70])

In [3]:
#gen_data = np.random.randint(1, 10, len(supply)*len(demand))
gen_data = ([1,2,6],
            [5,3,4],
            [4,5,7])

In [4]:
costs = np.reshape(gen_data, (-1, len(demand)))
my_costs = costs.copy

In [5]:
def illstr_table(dmnd, sply):
    i_table = pd.DataFrame(costs, columns=dmnd, index=sply)
    for i in range(len(dmnd)):
        i_table.rename(columns={'0': str(dmnd[i])})
    return i_table

# Транспортная задача

## Постановка транспортной задачи

<br>&#8195;&#8195;Транспортная задача - это математическая задача линейного программирования специального
вида о поиске оптимального распределения однородных объектов с минимизацией затрат на перемещение.
     
<br>&#8195;&#8195;Имеется ${m}$ поставщиков $A_1$, $A_2$, ..., $A_m$, у которых сосредоточены запасы одного и того же груза в количестве $a_1$, $a_2$, ..., $a_m$ единиц, соответственно. Этот груз нужно доставить ${n}$ потребителям $B_1$, $B_2$, ..., $B_n$, заказавшим $b_1$, $b_2$, ..., $b_n$ единиц этого груза, соответственно. Известны также все тарифы перевозок груза $c_{ij}$ (стоимость перевозок единицы груза) от поставщика $A_i$ к потребителю $B_j$. Требуется составить такой план перевозок, при котором общая стоимость всех перевозок была бы минимальной.

In [6]:
illstr_table(demand, supply) #illustration

,30,40,30
15,1,2,6
15,5,3,4
70,4,5,7


## Решение транспортной задачи
<br>&#8195;&#8195;Решение транспортной задачи начинается с выяснения вопроса о том, является ли задача открытой или закрытой.
    
<br>&#8195;&#8195;Если задача является открытой, то необходимо провести процедуру закрытия задачи. С этой целью при a<b добавляем фиктивного поставщика $A'_{m+1}$ с запасом груза $a'_{m+1}=b-a$. Если же a>b, то добавляем фиктивного потребителя $B'_{n+1}$ с заказом груза $b'_{n+1}=a-b$.
    
<br>&#8195;&#8195;В обоих случаях соответствующие фиктивным объектам тарифы перевозок $c'_{ij}$ полагаем равными нулю. В результате суммарная стоимость перевозок Z не изменяется.

In [7]:
def check_balanced(dmnd, sply):
    return "Открытая" if sum(dmnd)==sum(sply) else "Закрытая"

In [8]:
check_balanced(demand, supply)

'Открытая'

Транспортная задача относится к задачам линейного программирования, и ее можно было бы решить симплекс-методом. Но поскольку система огра- ничений транспортной задачи проще, чем система ограничений ОЗЛП, то это дает возможность вместо использования объемных симплекс-таблиц применить более удобный метод, который состоит из следующих этапов:
    
    1. Составление первоначального плана перевозок.
    
    2. Последовательные улучшения плана перевозок (перераспределение
поставок) до тех пор, пока план перевозок не станет оптимальным.

In [9]:
display(illstr_table(demand, supply).replace(costs, '-'))

,30,40,30
15,-,-,-
15,-,-,-
70,-,-,-


In [10]:
def change_row_v_for_illstr(t, i):
    for j in range(len(t.columns)):
        if t.values[i][j] == '-': 
            t.iat[i,j] = '+'

def change_column_v_for_illstr(t, j):
    for i in range(len(t.index)):
         if t.values[i][j] == '-': 
                t.iat[i,j] = '+'
                
def solution(dmnd, sply):  
    fg = 0
    cap_coord = []
    table = illstr_table(dmnd, sply).replace(costs, '-')
    display(table)
    for i in range(len(sply)):
        for j in range(len(dmnd)):
            if j != fg:
                continue
            fg += 1
            cell_v = min(sply[i], dmnd[j])
            sply[i] -= cell_v
            dmnd[j] -= cell_v
            table.index = sply
            table.columns = dmnd
            cap_coord.append([cell_v, [i,j]])
            if sply[i] == 0:
                fg = j
                change_row_v_for_illstr(table, i)
                table.iat[i,j] = cell_v
                display(table)
                break
            else: 
                change_column_v_for_illstr(table, j)
                table.iat[i,j] = cell_v
                display(table)
    return cap_coord

In [11]:
def get_ans(cap_coord, csts):
    summ=0
    route = []
    for cap, coord in cap_coord:
        summ += csts[coord[0]][coord[1]]*cap
        route.append(coord)
    return summ, route

## Составление первоначального плана перевозок с помощью метода северо-западного угла
<br>&#8195;&#8195;Составление первоначального плана перевозок начнем с перевозки запасов поставщика $A_1$. Будем за счет его запасов максимально возможно удовлетворять заказы сначала потребителя $B_1$, затем $B_2$ и так далее. Таким образом, мы будем заполнять таблицу, начиная с клетки (1,1), и двигаться вправо по строке до тех пор, пока остаток запасов поставщика $A_1$ не окажется меньше заказа очередного потребителя. Для выполнения этого заказа используем остатки запаса первого поставщика, а недостающую часть добавим из запасов поставщика $A_2$, то есть переместимся на следующую строку таблицы по столбцу, соответствующему указанному потребителю. Далее аналогичным образом распределим запасы поставщика $A_2$, затем $A_3$ и так далее.

In [12]:
cost, route = get_ans(solution(demand, supply), costs)
print("Cost: {}, \nRoute: {}".format(cost, route))

,30,40,30
15,-,-,-
15,-,-,-
70,-,-,-


,15,40,30
0,15,+,+
15,-,-,-
70,-,-,-


,0,40,30
0,15,+,+
0,15,+,+
70,-,-,-


,0,40,30
0,15,+,+
0,15,+,+
70,0,-,-


,0,0,30
0,15,+,+
0,15,+,+
30,0,40,-


,0,0,0
0,15,+,+
0,15,+,+
0,0,40,30


Cost: 500, 
Route: [[0, 0], [1, 0], [2, 0], [2, 1], [2, 2]]


## Составление первоначального плана перевозок с помощью метода наименьшей стоимости
<br>&#8195;&#8195;Построение плана начнем с клетки с наименьшим тарифом перевозок. При наличии нескольких клеток с одинаковыми тарифами выберем любую из них. Пусть это будет клетка (i, j) . Запишем в эту клетку элемент $x_{ij}$=min($a_i$,$b_j$). Если $a_i$<$b_j$, то запасы поставщика $A_i$ исчерпаны, а потребителю $B_j$ требуется еще $b'_j=b_j-a_i$ единиц груза. Поэтому, не принимая более во внимание i-ю строку, снова ищем клетку с наименьшей стоимостью перевозок и заполняем ее с учетом изменившихся потребностей. В случае $a_i$>$b_j$ из рассмотрения исключается j-й столбец, а запасы $A_i$ полагаются равными $a'_i=a_i-b_j$. Продолжаем этот процесс до тех пор, пока все запасы не будут исчерпаны, а все потребности удовлетворены.
<br>&#8195;&#8195;Необходимо отметить, что при наличии в таблице клеток с одинаковыми тарифами, планы, полученные с помощью этого метода, могут быть разными, однако они, несомненно, ближе к оптимальному плану, чем план, составленный по методу северо-западного угла.

In [13]:
def find(target,ToSearch):
    x=[(i, t.index(target))
    for i, t in enumerate(ToSearch)
    if target in t]
    return x

def MatMin(supplyinfo,demandinfo,costinfo):
    C=np.matrix(costinfo.copy())
    sn=len(supplyinfo)
    dn=len(demandinfo)
    sol=np.zeros((sn,dn))
    costs=list(costinfo)
    supply=supplyinfo.copy()
    demand=demandinfo.copy()
    count=0
    while (sum(supply)+sum(demand)) != 0 and count<20:
        count=count+1
        Min=[]
        for a in range(sn):
            row=costs[a]
            Row=[]
            for b in range(dn):
                if type(row[b])==int:
                    Row.append(row[b])
                 
                if b==dn-1 and len(Row)<=0:
                    continue
                if b==dn-1:
                    Min.append(min(Row))
        if len(Min)==0:
            print(sol)

            z=0
            for k in range(sn):
                for l in range(dn):
                    z=z+C[k,l]*sol[k,l]
            print('Стоимость', int(z))
            return sol
        smallest=min(Min)
        location=find(smallest,costs)
        if len(location)==1: 
            location=find(smallest,costs)[0]
            costs[location[0]][location[1]]='x'
            if supply[location[0]]>=demand[location[1]]:   
                sol[location[0]][location[1]]=demand[location[1]]
                demand[location[0]]=supply[location[0]]-demand[location[1]]
                demand[location[1]]=0
            else:
                sol[location[0]][location[1]]=supply[location[0]]
                demand[location[1]]=demand[location[1]]-supply[location[0]]
                supply[location[0]]=0 
        else: 
            locations=find(smallest,costs)
            l=len(locations)
            m=[]
            for a in range(l):
                L=locations[a]
                s=L[1]
                d=L[0]
                if s>=d:
                    m.append(s)
                else:
                    m.append(d)
            M=max(m)
            loc=m.index(M)
            location=locations[loc]
            costs[location[0]][location[1]]='x'
            if supply[location[0]]>=demand[location[1]]:                
                sol[location[0]][location[1]]=demand[location[1]]
                supply[location[0]]=supply[location[0]]-demand[location[1]]
                demand[location[1]]=0     
            else:
                sol[location[0]][location[1]]=supply[location[0]]
                demand[location[1]]=demand[location[1]]-supply[location[0]]
                supply[location[0]]=0 
    feasabilitytest=np.count_nonzero(sol)
    print('Number of stepping stones =', feasabilitytest)
    if feasabilitytest==(sn+dn-1):
        print('Feasibility test passed')
    print(sol)

    z=0
    for k in range(sn):
        for l in range(dn):
            z=z+C[k,l]*sol[k,l]
    print('Cost is £', int(z),'for this solution.')
                
                

costs=[[1,2,6],[5,3,4],[4,5,7]]
MatMin([15,15,70],[30,40,30],costs)


[[15.  0.  0.]
 [ 0. 15.  0.]
 [15. 25. 45.]]
Стоимость 560


array([[15.,  0.,  0.],
       [ 0., 15.,  0.],
       [15., 25., 45.]])

In [14]:
# # Demans - A,B,C,D
# # W,X,Y - Supply
# # Cost matrix is given below
# import collections
# from collections import defaultdict
# costs  = {'W': {'A': 11, 'B': 13, 'C': 17, 'D': 14},
#           'X': {'A': 16, 'B': 18, 'C': 14, 'D': 10},
#           'Y': {'A': 21, 'B': 24, 'C': 13, 'D': 10}}
# demand = {'A': 200, 'B': 225, 'C': 275, 'D': 250}
# cols = sorted(iter(demand.keys()))
# supply = {'W': 250, 'X': 300, 'Y': 400}
# res = dict((k, defaultdict(int)) for k in costs)
# g = {}

# #g contains all the columns and rows names like - a,b,c,... and so on

# for x in supply:
#     g[x] = sorted(costs[x].iterkeys(), key=lambda g: costs[x][g])
# for x in demand:
#     g[x] = sorted(costs.iterkeys(), key=lambda g: costs[g][x])

# while g:
#     d = {}
#     for x in demand:
#         d[x] = (costs[g[x][1]][x] - costs[g[x][0]][x]) if len(g[x]) > 1 else costs[g[x][0]][x]
#     s = {}
#     for x in supply:
#         s[x] = (costs[x][g[x][1]] - costs[x][g[x][0]]) if len(g[x]) > 1 else costs[x][g[x][0]]
#     f = max(d, key=lambda n: d[n])
#     t = max(s, key=lambda n: s[n])
#     t, f = (f, g[f][0]) if d[f] > s[t] else (g[t][0], t)
#     v = min(supply[f], demand[t])
#     res[f][t] += v
#     demand[t] -= v
#     if demand[t] == 0:
#         for k, n in supply.iteritems():
#             if n != 0:
#                 g[k].remove(t)
#         del g[t]
#         del demand[t]
#     supply[f] -= v
#     if supply[f] == 0:
#         for k, n in demand.iteritems():
#             if n != 0:
#                 g[k].remove(f)
#         del g[f]
#         del supply[f]
 
# for n in cols:
#     print ("\t", n)
# print()
# cost = 0
# for g in sorted(costs):
#     print (g, "\t")
#     for n in cols:
#         y = res[g][n]
#         if y != 0:
#             print (y)
#         cost += y * costs[g][n]
#         print ("\t")
#     print()
# print ("\n\nTotal Cost = ", cost)